In [56]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
import re;

In [55]:
def load_data():
    diagnosis = pd.read_csv('/Users/ravishchawla/Downloads/DIAGNOSES_ICD.csv');
    notes = pd.read_csv('/Users/ravishchawla/Downloads/NOTEEVENTS.csv');
    icd9 = pd.read_csv('/Users/ravishchawla/Downloads/ICD10_Formatted.csv.txt', sep='|', names=['ICD9_CODE', 'ICD10_CODE', 'desc']);
    
    return diagnosis, notes, icd9;

In [3]:
diagnosis, notes, icd9 = load_data();

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [233]:
notes_small = notes.ix[len(notes) - 20000:];
notes_small.shape

(20000, 11)

In [234]:
#groups = notes.groupby('HADM_ID').apply(lambda row: list(set(row['TEXT'])));

In [235]:
def get_stopwords():
    stop_words = str();
    with open('nltk', 'r') as f:
        for line in f:
            stop_words = stop_words + '\\b' + line.strip() + '\\b' + '|';

    stop_words = stop_words[0:-1];

    return stop_words;

In [236]:
def clean_text(notes_df):
    stop_words = get_stopwords();
    
    notes_filtered = notes_df['TEXT'].apply(lambda row: re.sub("[^a-zA-Z0-9]", " ", row.lower()));

    notes_nostops = notes_filtered.apply(lambda row: re.sub(stop_words, ' ', row));
    notes_final = notes_nostops.apply(lambda row: " ".join(row.split()));
    
    notes_df = notes_df.drop('TEXT', axis=1);
    notes_df = notes_df.assign(TEXT = notes_final.values)
    
    return notes_df;

In [237]:
notes_filtered = clean_text(notes_small);


    for row in ['aa bb cc dd ee ff', 'bb bb cc bb cc', 'aa aa', 'dd dd']:
        test = re.sub('(\\baa\\b|\\bbb\\b)', ' ', row);
        print(test)

In [238]:
vectorizer = CountVectorizer();

notes_counts = vectorizer.fit_transform(notes_filtered['TEXT']);

In [239]:
tfidf_transformer = TfidfTransformer();
tfidf_counts = tfidf_transformer.fit_transform(notes_counts);

In [240]:
zero_counts = np.where(tfidf_counts.toarray() == 0)[0].shape
print(zero_counts);

(641761957,)


In [241]:
tfidf_counts.shape[0] * tfidf_counts[1].shape[1] - zero_counts[0]

1918043